In [3]:
import numpy as np

def jacobi_2x2(A):
    """
    Метод вращений Якоби для симметричной 2×2 матрицы.
    
    Параметры:
        A (np.ndarray): Симметричная матрица размера 2x2.

    Возвращает:
        tuple: (массив собственных значений, матрица собственных векторов)
    """
    assert A.shape == (2, 2)
    assert np.allclose(A, A.T), "Матрица должна быть симметричной"

    a, b = A[0, 0], A[0, 1]
    d = A[1, 1]

    if b == 0:
        ### Если матрица уже диагональна — собственные значения на диагонали
        eigvals = [a, d]
        eigvecs = np.eye(2)
    else:
        ### Вычисление вспомогательной переменной для определения угла поворота
        tau = (d - a) / (2 * b)
        ### Вычисление t = tan(theta), минимизирующего внеосевые элементы
        t = np.sign(tau) / (abs(tau) + np.sqrt(1 + tau ** 2))
        ### Косинус и синус угла поворота
        c = 1 / np.sqrt(1 + t ** 2)
        s = t * c

        ### Ортогональная матрица поворота
        R = np.array([[c, -s],
                      [s, c]])

        ### Диагонализация: R^T A R
        D = R.T @ A @ R
        eigvals = [D[0, 0], D[1, 1]]
        eigvecs = R

    return np.array(eigvals), eigvecs


def jacobi_method(A, eps=1e-10, max_iter=100):
    """
    Метод Якоби для нахождения всех собственных значений и векторов симметричной матрицы произвольного размера.

    Параметры:
        A (np.ndarray): Симметричная матрица n×n.
        eps (float): Порог для останова по малости внедиагональных элементов.
        max_iter (int): Максимальное число итераций.

    Возвращает:
        tuple: (массив собственных значений, матрица собственных векторов)
    """
    A = A.copy()
    n = A.shape[0]
    assert A.shape[1] == n
    assert np.allclose(A, A.T), "Матрица должна быть симметричной"

    ### Инициализация матрицы собственных векторов
    V = np.eye(n)

    for _ in range(max_iter):
        ### Поиск максимального внедиагонального элемента
        max_val = 0
        p = q = 0
        for i in range(n):
            for j in range(i+1, n):
                if abs(A[i, j]) > abs(max_val):
                    max_val = A[i, j]
                    p, q = i, j

        if abs(max_val) < eps:
            ### Условие выхода: все внедиагональные элементы малы
            break

        ### Вычисление параметров вращения
        if A[p, p] == A[q, q]:
            theta = np.pi / 4
            c = np.cos(theta)
            s = np.sin(theta)
        else:
            tau = (A[q, q] - A[p, p]) / (2 * A[p, q])
            t = np.sign(tau) / (abs(tau) + np.sqrt(1 + tau**2))
            c = 1 / np.sqrt(1 + t**2)
            s = t * c

        ### Обновление строк и столбцов матрицы A
        for i in range(n):
            if i != p and i != q:
                aip = A[i, p]
                aiq = A[i, q]
                ### Преобразование строк/столбцов i, связанных с p и q
                A[i, p] = A[p, i] = c * aip - s * aiq
                A[i, q] = A[q, i] = c * aiq + s * aip

        app = A[p, p]
        aqq = A[q, q]
        apq = A[p, q]
        ### Обновление диагональных элементов
        A[p, p] = c**2 * app - 2*s*c*apq + s**2 * aqq
        A[q, q] = s**2 * app + 2*s*c*apq + c**2 * aqq
        ### Зануление внедиагонального элемента
        A[p, q] = A[q, p] = 0.0

        ### Обновление матрицы собственных векторов
        for i in range(n):
            vip = V[i, p]
            viq = V[i, q]
            V[i, p] = c * vip - s * viq
            V[i, q] = s * vip + c * viq

    eigvals = np.diag(A)
    return eigvals, V


A = np.array([[4, 1, 2],
              [1, 3, 0],
              [2, 0, 2]], dtype=float)

eigvals, eigvecs = jacobi_method(A)
print("Собственные значения:", eigvals)
print("Собственные векторы:\n", eigvecs)


Собственные значения: [5.52891796 2.83255081 0.63853123]
Собственные векторы:
 [[ 0.8226726  -0.15351016 -0.54739786]
 [ 0.32530617  0.91675668  0.23180398]
 [ 0.46624638 -0.36877068  0.80412841]]
